<a href="https://colab.research.google.com/github/adigenova/uohdb/blob/main/code/SQL_VI_empresa_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base de datos empresa

## descargando la base de datos de github

In [2]:
#download the empresa database
!git clone https://github.com/jpwhite3/northwind-SQLite3.git

Cloning into 'northwind-SQLite3'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 35 (delta 4), reused 0 (delta 0), pack-reused 25
Unpacking objects: 100% (35/35), done.


In [3]:
!unzip /content/northwind-SQLite3/Northwind_large.sqlite.zip

Archive:  /content/northwind-SQLite3/Northwind_large.sqlite.zip
  inflating: Northwind_large.sqlite  
   creating: __MACOSX/
  inflating: __MACOSX/._Northwind_large.sqlite  


In [4]:
# load the SQL magic extension
# https://github.com/catherinedevlin/ipython-sql
# this extension allows us to connect to DBs and issue SQL command
%load_ext sql
# now we can use the magic extension to connect to our SQLite DB
# use %sql to write an inline SQL command
# use %%sql to write SQL commands in a cell
%sql sqlite:////content/Northwind_large.sqlite

'Connected: @/content/Northwind_large.sqlite'


### Display tables of the empresa database


In [4]:
%%sql
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';

 * sqlite:////content/Northwind_large.sqlite
Done.


name
Employee
Category
Customer
Shipper
Supplier
Order
Product
OrderDetail
CustomerCustomerDemo
CustomerDemographic


getting information about a particular table


In [5]:
%%sql 
PRAGMA table_info([Employee]);

 * sqlite:////content/Northwind_large.sqlite
Done.


cid,name,type,notnull,dflt_value,pk
0,Id,INTEGER,0,None,1
1,LastName,VARCHAR(8000),0,None,0
2,FirstName,VARCHAR(8000),0,None,0
3,Title,VARCHAR(8000),0,None,0
4,TitleOfCourtesy,VARCHAR(8000),0,None,0
5,BirthDate,VARCHAR(8000),0,None,0
6,HireDate,VARCHAR(8000),0,None,0
7,Address,VARCHAR(8000),0,None,0
8,City,VARCHAR(8000),0,None,0
9,Region,VARCHAR(8000),0,None,0


getting the SQL code that generate a table


## Create automatic documentation with [schemaspy](https://schemaspy.org/)

<img src="https://github.com/jpwhite3/northwind-SQLite3/blob/master/Northwind_ERD.png?raw=true" alt="ER Diagram" width="836" height="605" />


The relationship icon indicates a foreign key constraint and a one-to-many relationship. 




# SQL con empresa





## Consultas Avanzadas

In [18]:
# Determine si los empleados son 'experimentados' o 'Junior' en función de la fecha de contratación
%%sql
select LastName ||" "||FirstName "Nombre",HireDate as "Fecha Contrato", 
CASE WHEN HireDate < '2025-01-01' THEN 'Experimentado'
     WHEN HireDate >= '2025-01-01' THEN 'Junior'
     ELSE 'Otro' END AS "Estado"
FROM Employee

 * sqlite:////content/Northwind_large.sqlite
Done.


Nombre,Fecha Contrato,Estado
Davolio Nancy,2024-05-01,Experimentado
Fuller Andrew,2024-08-14,Experimentado
Leverling Janet,2024-04-01,Experimentado
Peacock Margaret,2025-05-03,Junior
Buchanan Steven,2025-10-17,Junior
Suyama Michael,2025-10-17,Junior
King Robert,2026-01-02,Junior
Callahan Laura,2026-03-05,Junior
Dodsworth Anne,2026-11-15,Junior


In [33]:
#Ordenando resultados basado en una condicion
%%sql
SELECT FirstName "Nombre", LastName "Apellido", 
BirthDate "Fecha Nacimiento", City "Ciudad", 
PostalCode "Codigo Postal" 
FROM Employee
ORDER BY 
CASE WHEN Birthdate > '1990-01-01' 
     THEN City 
     ELSE PostalCode 
    END

 * sqlite:////content/Northwind_large.sqlite
Done.


Nombre,Apellido,Fecha Nacimiento,Ciudad,Codigo Postal
Margaret,Peacock,1969-09-19,Redmond,98052
Nancy,Davolio,1980-12-08,Seattle,98122
Andrew,Fuller,1984-02-19,Tacoma,98401
Janet,Leverling,1995-08-30,Kirkland,98033
Michael,Suyama,1995-07-02,London,EC2 7JR
Robert,King,1992-05-29,London,RG1 9SP
Anne,Dodsworth,1998-01-27,London,WG2 7LT
Steven,Buchanan,1987-03-04,London,SW1 8JR
Laura,Callahan,1990-01-09,Seattle,98105


In [11]:
# Cual es la categoria de productos que vende mas unidades?
%%sql
SELECT c.CategoryName "Categoria", SUM(o.Quantity) AS "Total productos"
FROM OrderDetail o
JOIN Product p
ON o.ProductId = p.Id
JOIN Category c
ON p.Id = c.Id
GROUP BY 1
ORDER BY 2 Desc

 * sqlite:////content/Northwind_large.sqlite
Done.


Categoria,Total productos
Confections,209901
Condiments,209418
Grains/Cereals,206906
Meat/Poultry,205879
Produce,205437
Dairy Products,204578
Beverages,203353
Seafood,203223


In [12]:
# Cual es la categoria de productos que produce más dinero? 
%%sql
SELECT c.CategoryName "Categoria", 
SUM(o.Quantity) "Total Productos", 
SUM(o.Quantity * o.UnitPrice) "Total Vendido"
FROM OrderDetail o
JOIN Product p
ON o.ProductId = p.Id
JOIN Category c
ON p.Id = c.Id
GROUP BY 1
ORDER BY 3 Desc

 * sqlite:////content/Northwind_large.sqlite
Done.


Categoria,Total Productos,Total Vendido
Seafood,203223,8127800
Produce,205437,6162684
Meat/Poultry,205879,5146795
Dairy Products,204578,4500174.8
Grains/Cereals,206906,4416881.949999999
Condiments,209418,3977418.2
Beverages,203353,3659727.6
Confections,209901,2098810


In [18]:
#Determine los 10 productos que generan más vendidos inluyendo precio promedio, descuento promedio y precio venta
%%sql
SELECT p.Id "ID producto", p.ProductName "Nombre", c.CategoryName "Categoria", 
    AVG(o.UnitPrice) AS  "Precio Promedio", 
    AVG(o.Discount) AS  "Descuento Promedio", 
    SUM(o.UnitPrice*o.Quantity*(1-o.Discount)) AS "Venta Total"
 FROM Product p
  JOIN OrderDetail o
   ON p.Id = o.ProductId
    JOIN Category c
    ON p.CategoryId = c.Id
GROUP BY p.Id
ORDER BY "Venta Total" DESC limit 10;

 * sqlite:////content/Northwind_large.sqlite
Done.


ID producto,Nombre,Categoria,Precio Promedio,Descuento Promedio,Venta Total
38,Côte de Blaye,Beverages,263.4479249011858,0.0001358695652173913,54009228.735
29,Thüringer Rostbratwurst,Meat/Poultry,123.7597387695445,0.000250244140625,25791923.041999985
9,Mishi Kobe Niku,Meat/Poultry,96.9975924547034,6.205013651030033e-05,19899792.5
20,Sir Rodney's Marmalade,Confections,80.98998763906057,9.147095179233625e-05,16816617.36
18,Carnarvon Tigers,Seafood,62.49071667285555,0.0002661220448075257,12940859.375
59,Raclette Courdavault,Dairy Products,54.97427058968361,0.00031392342730518274,11358970.7
51,Manjimup Dried Apples,Produce,52.988335982393934,0.00024452867098667317,11083309.65
62,Tarte au sucre,Confections,49.28288043477547,0.00032114624505928847,10185139.069999997
43,Ipoh Coffee,Beverages,45.9898180029513,0.00017215937038858828,9548562.7
28,Rössle Sauerkraut,Produce,45.58516681135679,0.0001550291454793501,9334526.240000056


In [28]:
# Cuales son los productos más caros por proveedor?
%%sql
SELECT s.companyname "Nombre Proveedor", p.productname "Nombre Producto", p.unitprice "Precio Unidad"
FROM supplier as s
JOIN product as p ON s.id = p.supplierid
WHERE p.unitprice = (SELECT MAX(p2.unitprice) FROM product as p2 WHERE p2.supplierid = s.id)
ORDER BY p.unitprice DESC, s.companyname ASC, p.productname ASC
 limit 10

 * sqlite:////content/Northwind_large.sqlite
Done.


Nombre Proveedor,Nombre Producto,Precio Unidad
Aux joyeux ecclésiastiques,Côte de Blaye,263.5
Plutzer Lebensmittelgroßmärkte AG,Thüringer Rostbratwurst,123.79
Tokyo Traders,Mishi Kobe Niku,97
"Specialty Biscuits, Ltd.",Sir Rodney's Marmalade,81
"Pavlova, Ltd.",Carnarvon Tigers,62.5
Gai pâturage,Raclette Courdavault,55
"G'day, Mate",Manjimup Dried Apples,53
Forêts d'érables,Tarte au sucre,49.3
Leka Trading,Ipoh Coffee,46
Heli Süßwaren GmbH & Co. KG,Schoggi Schokolade,43.9


In [32]:
%%sql
select MAX(unitprice), product.id, supplier.id from product, supplier where product.id=supplier.id;

 * sqlite:////content/Northwind_large.sqlite
Done.


MAX(unitprice),Id,Id_1
123.79,29,29


In [56]:
%%sql
select t.*,  t."Venta Total"-t."Venta Descuento Total" as "Descuento Total" from (
select o.ProductID "ID producto", 
sum(Quantity) "Unidades Total", 
sum(Quantity*UnitPrice) "Venta Total", 
sum(Quantity*UnitPrice*(1-Discount)) "Venta Descuento Total" from OrderDetail as o
Group by  o.ProductId
ORDER by 4 DESC ) as t
limit 10;

 * sqlite:////content/Northwind_large.sqlite
Done.


ID producto,Unidades Total,Venta Total,Venta Descuento Total,Descuento Total
38,205055,54017816.2,54009228.735,8587.465000003576
29,208449,25799290.769999985,25791923.041999985,7367.728000000119
9,205173,19901393.0,19899792.5,1600.5
20,207647,16817689.8,16816617.36,1072.440000001341
18,207126,12943675.0,12940859.375,2815.625
59,206729,11364111,11358970.7,5140.300000000745
51,209216,11086232.6,11083309.65,2922.949999999255
62,206720,10187731.999999998,10185139.069999997,2592.9300000015646
43,207646,9550115.2,9548562.7,1552.5
28,204781,9335695.200000055,9334526.240000056,1168.9599999990314


In [50]:
%%sql
PRAGMA table_info([OrderDetail]);

 * sqlite:////content/Northwind_large.sqlite
Done.


cid,name,type,notnull,dflt_value,pk
0,Id,VARCHAR(8000),0,None,1
1,OrderId,INTEGER,1,None,0
2,ProductId,INTEGER,1,None,0
3,UnitPrice,DECIMAL,1,None,0
4,Quantity,INTEGER,1,None,0
5,Discount,DOUBLE,1,None,0
